In [1]:
# Drill-Funktionen importieren
import drill as drill
import modelTuning as modelTuning
import prepareData as prepareData
import validation as validation
import createModel as createModel

In [2]:
import requests
import numpy as np
import json
import pandas as pd
import seaborn as sns
import datetime as dt
from math import sqrt

import matplotlib.pyplot as mplt
import plotly.express as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, classification_report
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn import metrics
from sklearn import preprocessing
from sklearn.decomposition import PCA

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import SpatialDropout1D

pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
#rooms = {'0':'Daniel', '1':'Felix N#2','2':'Calvin','3':'bigDataLab',
#         '4':'FelixAkku', '7':'Lukasbuero','9':'Felix B. #1','10':'Felix N #1'}

#for room in rooms:
#    print('Processing ' + rooms[room])
#    df = pd.DataFrame
#    df = drill.get_PIR_data(room=room, presence=True)
#    if not df.empty:
#        df.to_json('data\\' + room + '.json')

In [4]:
#df = drill.get_PIR_data(room='H215', presence=True)
#df.to_json('data\H215.json')
#df = drill.get_PIR_data(room='H216', presence=True)
#df.to_json('data\H216.json')
df = drill.get_PIR_data(room='H217', presence=True)
#df.to_json('data\H217.json')
#df = drill.get_PIR_data('dfs', False)
#df.to_json('data\test.json')

<Response [200]>


In [5]:
df.head()

,timestamp,upload_date,measurement_count,version,room,serial_number,co2_ppm,temperature_celsius,relative_humidity_percent,presence,battery
0,2021-08-15 02:00:00,1628985600000,582075,1.2.8,Elsen,s_8caab57cff7e_309633,389,22.5,59.500000,0,1
1,2021-08-15 02:00:06,1628985600000,582076,1.2.8,Elsen,s_8caab57cff7e_309633,389,22.5,59.599998,0,1
2,2021-08-15 02:00:12,1628985600000,582077,1.2.8,Elsen,s_8caab57cff7e_309633,388,22.5,59.599998,0,1
3,2021-08-15 02:00:17,1628985600000,582078,1.2.8,Elsen,s_8caab57cff7e_309633,388,22.5,59.599998,0,1
4,2021-08-15 02:00:23,1628985600000,582079,1.2.8,Elsen,s_8caab57cff7e_309633,389,22.5,59.500000,0,2


In [5]:
df = pd.read_json('data\\H217_.json') 
df_comp = pd.read_json('data\\living_room.json')

In [6]:
#plt.scatter(df, x='timestamp', y='co2_ppm', color='presence')

In [7]:
df_new = prepareData.preProcessDataset(df)
df_comp = prepareData.preProcessDataset(df_comp)

[]
[]


In [8]:
#plt.scatter(df_new, x='timestamp', y='co2_ppm', color='presence')

In [9]:
# timestamp, presence, temperatur und humidity entfernen
# temp/humid erhoehen Genauigkeit deutlich, da relativ unverlaesslich -> von zu vielen aeusseren Faktoren abhaengig
df_timestamp = df_new['timestamp']
y_presence = df_new['presence']

X_presence = df_new.drop(['timestamp', 'second', 'presence', 'temperature_celsius', 'relative_humidity_percent'], axis=1)
X_presence, scaler = prepareData.normalize_min_max(X_presence)
#X_presence = X_presence.drop(['second_sin', 'second_cos'], axis=1)

Xtrain, Xtest, ytrain, ytest = train_test_split(X_presence, y_presence, test_size=0.2, random_state=1, shuffle=False)

In [10]:
# shift des trainings-sets um n minuten in die Vergangenheit
# -> Test ob Model auch in die Zukunft Erwartungen treffen kann
#ytrain = ytrain.shift(-5)
#ytrain = ytrain.replace(np.nan, 0)

In [16]:
# Create Classifiers and save to disk
#modelClass = RandomForestClassifier()
#modelClass.fit(Xtrain, ytrain)
#modelTuning.saveModel(modelClass, 'models\RFC.mod')

#modelClass = SVC()
#modelClass.fit(Xtrain, ytrain)
#modelTuning.saveModel(modelClass, 'models\SVC.mod') 

#modelClass = GradientBoostingClassifier()
#modelClass.fit(Xtrain, ytrain)
#modelTuning.saveModel(modelClass, 'models\GBC.mod') 

#modelClass = BaggingClassifier()
#modelClass.fit(Xtrain, ytrain)
#modelTuning.saveModel(modelClass, 'models\BC.mod')

modelLogReg = LogisticRegression()
modelLogReg.fit(Xtrain, ytrain)
modelTuning.saveModel(modelLogReg, 'models\LR.mod')

In [17]:
model = modelTuning.loadModel('models\LR.mod')
ypred = model.predict(Xtest)
rf_base_accuracy = modelTuning.evaluateClassifier(model, X_presence, y_presence)
print(rf_base_accuracy)

0.8861481975967959


In [18]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

         0.0       0.94      0.92      0.93      9498
         1.0       0.72      0.76      0.74      2486

    accuracy                           0.89     11984
   macro avg       0.83      0.84      0.83     11984
weighted avg       0.89      0.89      0.89     11984



In [19]:
df_valid_class = Xtest.copy()
df_valid_class['timestamp'] = df_timestamp
df_valid_class['prediction'] = ypred
df_valid_class['co2_ppm'] = df_new['co2_ppm']

plt.scatter(df_valid_class, x='timestamp', y='co2_ppm', color='prediction')

In [ ]:
# plot feature-importance
if (model.feature_importances_.any):
    feature_imp = pd.Series(modelClass.feature_importances_,index=Xtrain.columns).sort_values(ascending=False)
    # Creating a bar plot
    ax = sns.barplot(x=feature_imp, y=feature_imp.index)
    ax.set(xlabel='Importance', ylabel='Feature')

In [ ]:
modelPCA = PCA(n_components=2)
modelPCA.fit(X_presence)
X_2D = modelPCA.transform(X_presence)

df_new['PCA1'] = X_2D[:, 0]
df_new['PCA2'] = X_2D[:, 1]
sns.lmplot(x="PCA1", y="PCA2", hue='presence', col='presence', data=df_new, fit_reg=False)

In [ ]:
# unsupervised learning: Clustering
modelKM = GaussianMixture(n_components=2, covariance_type='full')
modelKM.fit(X_presence)
y_ggm = model.predict(X_presence)

df_new['cluster'] = y_ggm
sns.lmplot(x="PCA1", y="PCA2", data=df_new, hue='presence', col='cluster', fit_reg=False)

In [ ]:
# falsche Werte im November korrigieren
df_comp.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
df_timestamp_comp = df_comp['timestamp']
df_comp.drop(['timestamp', 'second'], axis=1, inplace=True)#, 'second', 'presence', 'temperature_celsius', 'relative_humidity_percent'], axis=1, inplace=True)
#df_comp.drop(['second_sin', 'second_cos'], axis=1, inplace=True)
ypred_comp = model.predict(df_comp)

df_valid_comp = df_comp.copy()
df_valid_comp['timestamp'] = df_timestamp_comp
df_valid_comp['prediction'] = ypred_comp
df_valid_comp['co2_ppm'] = df_comp['co2_ppm']

In [ ]:
plt.scatter(df_valid_comp, x='timestamp', y='co2_ppm', color='prediction')

In [ ]:
#modelTuning.parameterTuning('RF', Xtrain, ytrain, Xtest, ytest, X_presence, y_presence)
#modelTuning.parameterTuning('GB', Xtrain, ytrain, Xtest, ytest, X_presence, y_presence)
#modelTuning.parameterTuning('SVC', Xtrain, ytrain, Xtest, ytest, X_presence, y_presence)
#modelTuning.parameterTuning('KNN', Xtrain, ytrain, Xtest, ytest, X_presence, y_presence)

In [ ]:
X_presence, scaler = prepareData.normalize_min_max(X_presence)
#Xtest, _ = normalize_min_max(Xtrain, scaler)

In [ ]:
timesteps = 25
X, y = prepareData.reshape_data_for_LSTM(X_presence, y_presence, timesteps)

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=1, shuffle=False)

In [ ]:
model = createModel.createLSTM(Xtrain)
model.summary()

In [ ]:
history_lstm = model.fit(Xtrain, ytrain, epochs=10, batch_size=1, verbose=1,validation_data= (Xtest, ytest))

In [ ]:
model = createModel.createNN(Xtrain)
model.summary()

In [ ]:
history = model.fit(Xtrain, ytrain, epochs=100, validation_split=0.2)

In [ ]:
val_loss, val_acc = model.evaluate(X, y)
print(val_loss, val_acc)

In [ ]:
mplt = validation.plotHistoryAccuracy(history_lstm)
mplt.show()

In [ ]:
mplt = validation.plotHistoryLoss(history_lstm)
mplt.show()

In [ ]:
mplt = validation.plotHistoryAccuracy(history)
mplt.show()

In [ ]:
mplt = validation.plotHistoryLoss(history)
mplt.show()